# Part 3: Data Analytics

This notebook performs data analytics on:
1. CSV file from Part 1: `pr.data.0.Current` (BLS time-series data)
2. JSON file from Part 2: Population data from API

## Tasks:
1. Load both files as dataframes
2. Calculate mean and standard deviation of annual US population (2013-2018)
3. Find best year for each series_id (year with max sum of values across quarters)
4. Generate report for series_id = PRS30006032, period = Q01 with population data

In [22]:
# Import required libraries
import boto3
import pandas as pd
import json
import io
from datetime import datetime
import numpy as np

# AWS Setup
s3 = boto3.client('s3')
bucket_name = 'bls-dataset-sync2'

print("Libraries imported successfully!")

Libraries imported successfully!


## Step 1: Load Data from S3

In [23]:
# Function to download and load CSV file from S3
def load_csv_from_s3(bucket, key):
    """Download CSV file from S3 and return as pandas DataFrame"""
    try:
        response = s3.get_object(Bucket=bucket, Key=key)
        csv_content = response['Body'].read().decode('utf-8')
        df = pd.read_csv(io.StringIO(csv_content), sep='\t')  # BLS files are tab-separated
        return df
    except Exception as e:
        print(f"Error loading CSV from S3: {e}")
        return None

# Function to download and load JSON file from S3
def load_json_from_s3(bucket, key):
    """Download JSON file from S3 and return as pandas DataFrame"""
    try:
        response = s3.get_object(Bucket=bucket, Key=key)
        json_content = response['Body'].read().decode('utf-8')
        data = json.loads(json_content)
        # Convert JSON to DataFrame
        df = pd.DataFrame(data.get('data', []))
        return df
    except Exception as e:
        print(f"Error loading JSON from S3: {e}")
        return None

# List files in S3 to find the correct filenames
print("Listing files in S3 bucket...")
try:
    response = s3.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in response:
        files = [obj['Key'] for obj in response['Contents']]
        print(f"Found {len(files)} files in S3:")
        for file in sorted(files):
            print(f"  - {file}")
        
        # Find the CSV file (pr.data.0.Current)
        csv_file = None
        for file in files:
            if 'pr.data.0.Current' in file or file == 'pr.data.0.Current':
                csv_file = file
                break
        
        # Find the most recent population JSON file
        json_files = [f for f in files if f.startswith('population_data_') and f.endswith('.json')]
        json_file = sorted(json_files)[-1] if json_files else None
        
        print(f"\nUsing CSV file: {csv_file}")
        print(f"Using JSON file: {json_file}")
    else:
        print("No files found in S3 bucket")
        csv_file = 'pr.data.0.Current'
        json_file = None
except Exception as e:
    print(f"Error listing S3 files: {e}")
    csv_file = 'pr.data.0.Current'
    json_file = None

Listing files in S3 bucket...
Found 13 files in S3:
  - /pub/time.series/pr/pr.class
  - /pub/time.series/pr/pr.contacts
  - /pub/time.series/pr/pr.data.0.Current
  - /pub/time.series/pr/pr.data.1.AllData
  - /pub/time.series/pr/pr.duration
  - /pub/time.series/pr/pr.footnote
  - /pub/time.series/pr/pr.measure
  - /pub/time.series/pr/pr.period
  - /pub/time.series/pr/pr.seasonal
  - /pub/time.series/pr/pr.sector
  - /pub/time.series/pr/pr.series
  - /pub/time.series/pr/pr.txt
  - population_data_20260204_000208.json

Using CSV file: /pub/time.series/pr/pr.data.0.Current
Using JSON file: population_data_20260204_000208.json


In [24]:
# Load the CSV file (Part 1 data)
if csv_file:
    print(f"Loading CSV file: {csv_file}")
    df_bls = load_csv_from_s3(bucket_name, csv_file)
    if df_bls is not None:
        print(f"BLS DataFrame loaded successfully!")
        print(f"Shape: {df_bls.shape}")
        print(f"Columns: {list(df_bls.columns)}")
        print("\nFirst few rows:")
        print(df_bls.head())
        print("\nData types:")
        print(df_bls.dtypes)
    else:
        print("Failed to load BLS CSV file")
else:
    print("CSV file not found in S3")
    df_bls = None

Loading CSV file: /pub/time.series/pr/pr.data.0.Current
BLS DataFrame loaded successfully!
Shape: (37521, 5)
Columns: ['series_id        ', 'year', 'period', '       value', 'footnote_codes']

First few rows:
   series_id          year period         value footnote_codes
0  PRS30006011        1995    Q01           2.6            NaN
1  PRS30006011        1995    Q02           2.1            NaN
2  PRS30006011        1995    Q03           0.9            NaN
3  PRS30006011        1995    Q04           0.1            NaN
4  PRS30006011        1995    Q05           1.4            NaN

Data types:
series_id                str
year                   int64
period                   str
       value         float64
footnote_codes           str
dtype: object


In [25]:
# Load the JSON file (Part 2 data)
if json_file:
    print(f"Loading JSON file: {json_file}")
    df_population = load_json_from_s3(bucket_name, json_file)
    if df_population is not None:
        print(f"Population DataFrame loaded successfully!")
        print(f"Shape: {df_population.shape}")
        print(f"Columns: {list(df_population.columns)}")
        print("\nFirst few rows:")
        print(df_population.head())
        print("\nData types:")
        print(df_population.dtypes)
    else:
        print("Failed to load population JSON file")
else:
    print("JSON file not found in S3")
    df_population = None

Loading JSON file: population_data_20260204_000208.json
Population DataFrame loaded successfully!
Shape: (10, 4)
Columns: ['Nation ID', 'Nation', 'Year', 'Population']

First few rows:
  Nation ID         Nation  Year   Population
0   01000US  United States  2013  316128839.0
1   01000US  United States  2014  318857056.0
2   01000US  United States  2015  321418821.0
3   01000US  United States  2016  323127515.0
4   01000US  United States  2017  325719178.0

Data types:
Nation ID         str
Nation            str
Year            int64
Population    float64
dtype: object


## Step 2: Data Cleaning and Preparation

In [26]:
# IMPORTANT: Ensure column names are cleaned before proceeding
# This cell should be run after the data cleaning cell above
if df_bls is not None:
    # Strip column names if not already done
    original_cols = list(df_bls.columns)
    df_bls.columns = df_bls.columns.str.strip()
    if original_cols != list(df_bls.columns):
        print("Column names have been cleaned!")
        print(f"Before: {original_cols}")
        print(f"After: {list(df_bls.columns)}")
    else:
        print("Column names are already clean.")
        print(f"Columns: {list(df_bls.columns)}")
    
    # Verify required columns exist
    required_cols = ['series_id', 'year', 'period', 'value']
    missing = [c for c in required_cols if c not in df_bls.columns]
    if missing:
        print(f"WARNING: Missing columns: {missing}")
        print(f"Available columns: {list(df_bls.columns)}")
    else:
        print("✓ All required columns are present!")
else:
    print("BLS DataFrame is None")

Column names have been cleaned!
Before: ['series_id        ', 'year', 'period', '       value', 'footnote_codes']
After: ['series_id', 'year', 'period', 'value', 'footnote_codes']
✓ All required columns are present!


In [27]:
# Clean BLS data: trim whitespaces from column names and string columns
if df_bls is not None:
    print("Cleaning BLS data...")
    
    # First, strip whitespaces from column names
    df_bls.columns = df_bls.columns.str.strip()
    print(f"Column names after stripping: {list(df_bls.columns)}")
    
    # Trim whitespaces from all string columns
    string_columns = df_bls.select_dtypes(include=['object']).columns
    for col in string_columns:
        df_bls[col] = df_bls[col].str.strip()
    
    # Ensure numeric columns are numeric
    if 'value' in df_bls.columns:
        df_bls['value'] = pd.to_numeric(df_bls['value'], errors='coerce')
    if 'year' in df_bls.columns:
        df_bls['year'] = pd.to_numeric(df_bls['year'], errors='coerce')
    
    print("BLS data cleaned!")
    print(f"Sample data after cleaning:")
    print(df_bls.head())
    print(f"\nFinal column names: {list(df_bls.columns)}")
else:
    print("BLS DataFrame is None, skipping cleaning")

Cleaning BLS data...
Column names after stripping: ['series_id', 'year', 'period', 'value', 'footnote_codes']
BLS data cleaned!
Sample data after cleaning:
     series_id  year period  value footnote_codes
0  PRS30006011  1995    Q01    2.6            NaN
1  PRS30006011  1995    Q02    2.1            NaN
2  PRS30006011  1995    Q03    0.9            NaN
3  PRS30006011  1995    Q04    0.1            NaN
4  PRS30006011  1995    Q05    1.4            NaN

Final column names: ['series_id', 'year', 'period', 'value', 'footnote_codes']


C:\Users\spectre\AppData\Local\Temp\ipykernel_5160\3777628433.py:10: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  string_columns = df_bls.select_dtypes(include=['object']).columns


In [28]:
# Clean and prepare population data
if df_population is not None:
    print("Cleaning population data...")
    
    # Trim whitespaces from string columns
    string_columns = df_population.select_dtypes(include=['object']).columns
    for col in string_columns:
        df_population[col] = df_population[col].str.strip()
    
    # Identify the Year column (could be 'Year', 'year', etc.)
    year_col = None
    pop_col = None
    
    for col in df_population.columns:
        col_lower = col.lower()
        if 'year' in col_lower:
            year_col = col
        if 'population' in col_lower:
            pop_col = col
    
    print(f"Year column: {year_col}")
    print(f"Population column: {pop_col}")
    
    # Ensure Year is numeric
    if year_col:
        df_population[year_col] = pd.to_numeric(df_population[year_col], errors='coerce')
    
    # Ensure Population is numeric
    if pop_col:
        df_population[pop_col] = pd.to_numeric(df_population[pop_col], errors='coerce')
    
    print("Population data cleaned!")
    print(f"Sample data after cleaning:")
    print(df_population.head())
else:
    print("Population DataFrame is None, skipping cleaning")

Cleaning population data...
Year column: Year
Population column: Population
Population data cleaned!
Sample data after cleaning:
  Nation ID         Nation  Year   Population
0   01000US  United States  2013  316128839.0
1   01000US  United States  2014  318857056.0
2   01000US  United States  2015  321418821.0
3   01000US  United States  2016  323127515.0
4   01000US  United States  2017  325719178.0


C:\Users\spectre\AppData\Local\Temp\ipykernel_5160\3178353700.py:6: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  string_columns = df_population.select_dtypes(include=['object']).columns


## Step 3: Population Statistics (2013-2018)

In [29]:
# Calculate mean and standard deviation of annual US population (2013-2018)
if df_population is not None and year_col and pop_col:
    # Filter data for years 2013-2018 (inclusive)
    population_filtered = df_population[
        (df_population[year_col] >= 2013) & 
        (df_population[year_col] <= 2018)
    ].copy()
    
    if len(population_filtered) > 0:
        # Get population values for the filtered years
        pop_values = population_filtered[pop_col].dropna()
        
        if len(pop_values) > 0:
            mean_population = pop_values.mean()
            std_population = pop_values.std()
            
            print("=" * 60)
            print("POPULATION STATISTICS (2013-2018)")
            print("=" * 60)
            print(f"Mean Annual US Population: {mean_population:,.2f}")
            print(f"Standard Deviation: {std_population:,.2f}")
            print(f"\nNumber of years in dataset: {len(pop_values)}")
            print(f"\nPopulation by year:")
            print(population_filtered[[year_col, pop_col]].sort_values(year_col))
            
            # Store results
            pop_stats = {
                'mean': mean_population,
                'std': std_population,
                'years': list(population_filtered[year_col].unique())
            }
        else:
            print("No valid population values found for years 2013-2018")
            pop_stats = None
    else:
        print("No data found for years 2013-2018")
        pop_stats = None
else:
    print("Population data not available or columns not identified")
    pop_stats = None

POPULATION STATISTICS (2013-2018)
Mean Annual US Population: 322,069,808.00
Standard Deviation: 4,158,441.04

Number of years in dataset: 6

Population by year:
   Year   Population
0  2013  316128839.0
1  2014  318857056.0
2  2015  321418821.0
3  2016  323127515.0
4  2017  325719178.0
5  2018  327167439.0


## Step 4: Best Year Report for Each Series ID

In [30]:
# Fix column name to match requirements (should be 'value', not 'summed_value')
# The report should have columns: series_id, year, value
if 'best_years' in globals() and best_years is not None:
    if 'summed_value' in best_years.columns:
        best_years = best_years.rename(columns={'summed_value': 'value'})
        best_years_report = best_years.copy()
        print("Column name fixed: 'summed_value' -> 'value'")
        print("\nFinal report with correct column names:")
        print(best_years[['series_id', 'year', 'value']].head(20))

In [31]:
# For each series_id, find the best year (year with max sum of values across all quarters)
if df_bls is not None:
    print("Calculating best year for each series_id...")
    
    # Ensure we have the required columns
    required_cols = ['series_id', 'year', 'period', 'value']
    missing_cols = [col for col in required_cols if col not in df_bls.columns]
    
    if missing_cols:
        print(f"Missing required columns: {missing_cols}")
        print(f"Available columns: {list(df_bls.columns)}")
    else:
        # Filter out rows with missing values
        df_clean = df_bls[required_cols].dropna()
        
        # Group by series_id and year, sum the values for all quarters in each year
        yearly_sums = df_clean.groupby(['series_id', 'year'])['value'].sum().reset_index()
        
        # For each series_id, find the year with the maximum sum
        best_years = yearly_sums.loc[
            yearly_sums.groupby('series_id')['value'].idxmax()
        ].copy()
        
        # Rename 'value' to indicate it's the summed value
        best_years = best_years.rename(columns={'value': 'summed_value'})
        
        # Sort by series_id for better readability
        best_years = best_years.sort_values('series_id').reset_index(drop=True)
        
        print("=" * 60)
        print("BEST YEAR REPORT FOR EACH SERIES ID")
        print("=" * 60)
        print(f"Total number of series: {len(best_years)}")
        print(f"\nFirst 20 rows:")
        print(best_years.head(20))
        print(f"\nLast 20 rows:")
        print(best_years.tail(20))
        
        # Display full report
        print(f"\n{'='*60}")
        print("FULL REPORT")
        print(f"{'='*60}")
        print(best_years.to_string(index=False))
        
        # Save the report
        best_years_report = best_years.copy()
else:
    print("BLS DataFrame is None, cannot generate best year report")
    best_years_report = None

Calculating best year for each series_id...
BEST YEAR REPORT FOR EACH SERIES ID
Total number of series: 282

First 20 rows:
      series_id  year  summed_value
0   PRS30006011  2022        20.500
1   PRS30006012  2022        17.100
2   PRS30006013  1998       705.895
3   PRS30006021  2010        17.700
4   PRS30006022  2010        12.400
5   PRS30006023  2014       503.216
6   PRS30006031  2022        20.500
7   PRS30006032  2021        17.100
8   PRS30006033  1998       702.672
9   PRS30006061  2022        34.500
10  PRS30006062  2021        29.800
11  PRS30006063  2024       643.539
12  PRS30006081  2021        24.500
13  PRS30006082  2021        24.500
14  PRS30006083  2022       131.294
15  PRS30006091  2002        43.400
16  PRS30006092  2002        44.300
17  PRS30006093  2013       514.158
18  PRS30006101  2020        33.100
19  PRS30006102  2020        35.700

Last 20 rows:
       series_id  year  summed_value
262  PRS88003142  2021        32.000
263  PRS88003143  2024       62

## Step 5: Combined Report (Series PRS30006032, Period Q01 with Population)

In [32]:
# Generate report for series_id = PRS30006032, period = Q01 with population data
if df_bls is not None:
    print("Generating combined report for PRS30006032, Q01...")
    
    # Filter for series_id = PRS30006032 and period = Q01
    # Trim whitespaces to ensure exact match
    filtered_bls = df_bls[
        (df_bls['series_id'].str.strip() == 'PRS30006032') & 
        (df_bls['period'].str.strip() == 'Q01')
    ].copy()
    
    if len(filtered_bls) > 0:
        print(f"Found {len(filtered_bls)} records for PRS30006032, Q01")
        
        # Prepare population data for joining
        if df_population is not None and year_col and pop_col:
            # Create a clean population dataframe with just Year and Population
            pop_df = df_population[[year_col, pop_col]].copy()
            pop_df = pop_df.rename(columns={year_col: 'year', pop_col: 'Population'})
            pop_df = pop_df.dropna()
            
            # Ensure year is integer for proper joining
            filtered_bls['year'] = filtered_bls['year'].astype(int)
            pop_df['year'] = pop_df['year'].astype(int)
            
            # Left join to add population data
            combined_report = filtered_bls.merge(
                pop_df,
                on='year',
                how='left'
            )
            
            # Select and order columns
            report_columns = ['series_id', 'year', 'period', 'value', 'Population']
            available_columns = [col for col in report_columns if col in combined_report.columns]
            combined_report = combined_report[available_columns]
            
            # Sort by year
            combined_report = combined_report.sort_values('year').reset_index(drop=True)
            
            print("=" * 60)
            print("COMBINED REPORT: PRS30006032, Q01 with Population")
            print("=" * 60)
            print(combined_report.to_string(index=False))
            
            # Show summary
            print(f"\nTotal records: {len(combined_report)}")
            records_with_pop = combined_report['Population'].notna().sum()
            print(f"Records with population data: {records_with_pop}")
            print(f"Records without population data: {len(combined_report) - records_with_pop}")
            
            # Save the report
            final_report = combined_report.copy()
        else:
            print("Population data not available for joining")
            # Still create report without population
            report_columns = ['series_id', 'year', 'period', 'value']
            available_columns = [col for col in report_columns if col in filtered_bls.columns]
            final_report = filtered_bls[available_columns].sort_values('year').reset_index(drop=True)
            print("=" * 60)
            print("COMBINED REPORT: PRS30006032, Q01 (without Population)")
            print("=" * 60)
            print(final_report.to_string(index=False))
    else:
        print("No records found for series_id = PRS30006032 and period = Q01")
        print(f"Available series_ids (sample): {df_bls['series_id'].unique()[:10]}")
        print(f"Available periods (sample): {df_bls['period'].unique()[:10]}")
        final_report = None
else:
    print("BLS DataFrame is None, cannot generate combined report")
    final_report = None

Generating combined report for PRS30006032, Q01...
Found 31 records for PRS30006032, Q01
COMBINED REPORT: PRS30006032, Q01 with Population
  series_id  year period  value  Population
PRS30006032  1995    Q01    0.0         NaN
PRS30006032  1996    Q01   -4.2         NaN
PRS30006032  1997    Q01    2.8         NaN
PRS30006032  1998    Q01    0.9         NaN
PRS30006032  1999    Q01   -4.1         NaN
PRS30006032  2000    Q01    0.5         NaN
PRS30006032  2001    Q01   -6.3         NaN
PRS30006032  2002    Q01   -6.6         NaN
PRS30006032  2003    Q01   -5.7         NaN
PRS30006032  2004    Q01    2.0         NaN
PRS30006032  2005    Q01   -0.5         NaN
PRS30006032  2006    Q01    1.8         NaN
PRS30006032  2007    Q01   -0.8         NaN
PRS30006032  2008    Q01   -3.5         NaN
PRS30006032  2009    Q01  -21.0         NaN
PRS30006032  2010    Q01    3.2         NaN
PRS30006032  2011    Q01    1.5         NaN
PRS30006032  2012    Q01    2.5         NaN
PRS30006032  2013    Q01 